In [40]:
import numpy as np

class DeepNet(object):
    # y ~ Ax + b
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.b = [ np.random.rand(y, 1) for y in sizes[1:] ]
        self.A = [ np.random.rand(y, x) for y, x in zip(sizes[1:], sizes[:-1]) ]


In [41]:
net = DeepNet([3,4,2,1])

In [42]:
net.b

[array([[0.89781173],
        [0.33775256],
        [0.55138773],
        [0.09376404]]),
 array([[0.34287926],
        [0.47474383]]),
 array([[0.78312131]])]

In [43]:
net.A

[array([[0.45093907, 0.46312342, 0.55976897],
        [0.58941615, 0.25782921, 0.33918561],
        [0.36733226, 0.64235887, 0.67072904],
        [0.70151616, 0.52449352, 0.41015912]]),
 array([[0.2147813 , 0.04842467, 0.30923646, 0.92145845],
        [0.12055936, 0.99720339, 0.08992844, 0.40838426]]),
 array([[0.49121158, 0.64383346]])]